 # Instructions:
 - For all cells marked with <font color="red">(CODE Needed)</font> below, replace `#CODE_HERE` with your solution


# 1. Load Spark Context (No CODE Needed)

In [31]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, desc, col, when, max
from functools import reduce

spark = SparkSession.builder.getOrCreate()

# 2. Load some printing functions (No CODE Needed)

In [32]:
#These are some printing helper functions we will use to make the output more clear
from pprint import pprint
def title(s):
    pprint("---- %s -----" %s)    
    
def see(s, v):
    pprint("---- %s -----" %s)
    pprint(v)

# 3. Load each file in an DF and add the year to the data (No CODE Needed)

In [33]:
def createOneYearDF(year):
    file = "../../data/flight-data/csv/%s"%year+"*"
    see("year:"+str(year), file)
    fileDF =spark.read.csv(file, header=True)
    arrDF = fileDF\
                .withColumn("year", lit(str(year)))\
                .withColumn("count", fileDF["count"].cast("int"))
    see("DF count for year:"+str(year), arrDF.count())
    title("DF elements for year:"+str(year))
    for arr in (arrDF.take(5)):
        print(arr)
    return arrDF

separateDFs  = list(map(createOneYearDF, range(2010,2016)))
see("arrays of DF", separateDFs)

'---- year:2010 -----'
'../../data/flight-data/csv/2010*'
'---- DF count for year:2010 -----'
255
'---- DF elements for year:2010 -----'
Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=1, year='2010')
Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=264, year='2010')
Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=69, year='2010')
Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=24, year='2010')
Row(DEST_COUNTRY_NAME='Equatorial Guinea', ORIGIN_COUNTRY_NAME='United States', count=1, year='2010')
'---- year:2011 -----'
'../../data/flight-data/csv/2011*'
'---- DF count for year:2011 -----'
255
'---- DF elements for year:2011 -----'
Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Saint Martin', count=2, year='2011')
Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Guinea', count=2, year='2011')
Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia'

# 4. Merge all DFs into one DF  (No CODE Needed)

In [34]:
allYearsDF = reduce(lambda df1, df2: df1.union(df2),separateDFs)
allYearsDF.createOrReplaceTempView("flightData")
see("allYearsDF count", allYearsDF.count())
see("allYearsDF sample", allYearsDF.rdd.takeSample(False, 10, 17))

title("SQL show(5)")
spark.sql("SELECT * FROM flightData").show(5)

'---- allYearsDF count -----'
1502
'---- allYearsDF sample -----'
[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Jamaica', count=714, year='2014'),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=29, year='2011'),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Samoa', count=28, year='2010'),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=24, year='2011'),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Saint Barthelemy', count=58, year='2013'),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Afghanistan', count=5, year='2012'),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Russia', count=151, year='2014'),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Thailand', count=4, year='2015'),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Sweden', count=68, year='2011'),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenad

# 5. Compute a DF containing all flights which departed from US  <font color="red">(CODE Needed)</font>
- Hint: make 1 `filter` transformation

In [38]:
sqlQuery = 'SELECT * FROM flightData WHERE ORIGIN_COUNTRY_NAME = "United States"'

spark.sql(sqlQuery).createOrReplaceTempView("fromUS")

title("fromUS count")
spark.sql("SELECT count(*) FROM fromUS").show()

title("fromUS show(10)")
spark.sql("SELECT * FROM fromUS").show(10)

'---- fromUS count -----'
+--------+
|count(1)|
+--------+
|     772|
+--------+

'---- fromUS show(10) -----'
+--------------------+-------------------+-----+----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|year|
+--------------------+-------------------+-----+----+
|               Egypt|      United States|   24|2010|
|   Equatorial Guinea|      United States|    1|2010|
|          Costa Rica|      United States|  477|2010|
|             Senegal|      United States|   29|2010|
|              Guyana|      United States|   17|2010|
|               Malta|      United States|    1|2010|
|             Bolivia|      United States|   46|2010|
|            Anguilla|      United States|   21|2010|
|Turks and Caicos ...|      United States|  136|2010|
|Saint Vincent and...|      United States|    1|2010|
+--------------------+-------------------+-----+----+
only showing top 10 rows



### Expectated output for step 5.

<pre>
'---- fromUS count -----'
+--------+
|count(1)|
+--------+
|     772|
+--------+

'---- fromUS show(10) -----'
+--------------------+-------------------+-----+----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|year|
+--------------------+-------------------+-----+----+
|               Egypt|      United States|   24|2010|
|   Equatorial Guinea|      United States|    1|2010|
|          Costa Rica|      United States|  477|2010|
|             Senegal|      United States|   29|2010|
|              Guyana|      United States|   17|2010|
|               Malta|      United States|    1|2010|
|             Bolivia|      United States|   46|2010|
|            Anguilla|      United States|   21|2010|
|Turks and Caicos ...|      United States|  136|2010|
|Saint Vincent and...|      United States|    1|2010|
+--------------------+-------------------+-----+----+
only showing top 10 rows
</pre>

# 6. Sum all the flights for the same destination and the same year <font color="red">(CODE Needed)</font>
- Hint: make 1 `groupBy` and 1 `sum` transformations

In [48]:
sqlQuery = 'SELECT DEST_COUNTRY_NAME, year, SUM(count) AS flight_count FROM fromUS GROUP BY DEST_COUNTRY_NAME, year'

spark.sql(sqlQuery).createOrReplaceTempView("groupedFlightData")

title("groupedFlightData count")
spark.sql("SELECT count(*) FROM groupedFlightData").show()

title("groupedFlightData show(10)")
spark.sql("SELECT * FROM groupedFlightData").show(10)

'---- groupedFlightData count -----'
+--------+
|count(1)|
+--------+
|     772|
+--------+

'---- groupedFlightData show(10) -----'
+-----------------+----+------------+
|DEST_COUNTRY_NAME|year|flight_count|
+-----------------+----+------------+
|      Saint Lucia|2011|         102|
|          Hungary|2014|           2|
|       Guadeloupe|2014|          43|
|          Nigeria|2014|          49|
|             Fiji|2015|          24|
|           Kuwait|2010|          26|
|   Cayman Islands|2011|         251|
|         Paraguay|2011|          85|
|          Austria|2011|          34|
|        Gibraltar|2014|           1|
+-----------------+----+------------+
only showing top 10 rows



### Exepected output for step  6
<pre>
'---- grouped count -----'
+--------+
|count(1)|
+--------+
|     772|
+--------+

'---- grouped show(10) -----'
+-----------------+----+------------+
|dest_country_name|year|flight_count|
+-----------------+----+------------+
|      Saint Lucia|2011|         102|
|          Hungary|2014|           2|
|       Guadeloupe|2014|          43|
|          Nigeria|2014|          49|
|             Fiji|2015|          24|
|           Kuwait|2010|          26|
|   Cayman Islands|2011|         251|
|         Paraguay|2011|          85|
|          Austria|2011|          34|
|        Gibraltar|2014|           1|
+-----------------+----+------------+
only showing top 10 rows
 </pre>

# 7. Sort the summed data by the highest sum <font color="red">(CODE Needed)</font>
- Hint: make 1 `orderBy` transformation


In [50]:
sqlQuery = 'SELECT * FROM groupedFlightData ORDER BY flight_count DESC'

spark.sql(sqlQuery).createOrReplaceTempView("sortedFlightData")

title("sortedFlightData count")
spark.sql("SELECT count(*) FROM sortedFlightData").show()

title("sortedFlightData show(10)")
spark.sql("SELECT * FROM sortedFlightData").show(10)

'---- sortedFlightData count -----'
+--------+
|count(1)|
+--------+
|     772|
+--------+

'---- sortedFlightData show(10) -----'
+-----------------+----+------------+
|DEST_COUNTRY_NAME|year|flight_count|
+-----------------+----+------------+
|    United States|2015|      370002|
|    United States|2014|      358354|
|    United States|2011|      352742|
|    United States|2010|      348113|
|    United States|2012|      347452|
|    United States|2013|      343132|
|           Canada|2011|        8514|
|           Canada|2015|        8399|
|           Canada|2010|        8271|
|           Canada|2012|        8034|
+-----------------+----+------------+
only showing top 10 rows



### Exepected output for step  7
<pre>
'---- sortedFlightData count -----'
+--------+
|count(1)|
+--------+
|     772|
+--------+

'---- sortedFlightData show(10) -----'
+-----------------+----+------------+
|dest_country_name|year|flight_count|
+-----------------+----+------------+
|    United States|2015|      370002|
|    United States|2014|      358354|
|    United States|2011|      352742|
|    United States|2010|      348113|
|    United States|2012|      347452|
|    United States|2013|      343132|
|           Canada|2011|        8514|
|           Canada|2015|        8399|
|           Canada|2010|        8271|
|           Canada|2012|        8034|
+-----------------+----+------------+
only showing top 10 rows
 </pre>

# 8. Compute the Sum of all Trips <font color="red">(CODE Needed)</font>
- Hint: make 1 `sum` transformations


In [51]:
sqlQuery = 'SELECT SUM(flight_count) AS total_count FROM sortedFlightData'

spark.sql(sqlQuery).createOrReplaceTempView("sumAllTrips")

title("sumAllTrips")
spark.sql("SELECT * FROM sumAllTrips").show(10)

'---- sumAllTrips -----'
+-----------+
|total_count|
+-----------+
|    2352430|
+-----------+



### Exepected output for step  8
<pre>
'---- sumAllTrips -----'
+-----------+
|total_count|
+-----------+
|    2352430|
+-----------+
 </pre>

# 9. Compute the Sum of Each Year's Trip <font color="red">(CODE Needed)</font>
- Hint: make 1 `groupBy` , 1 `sum` transformations


In [53]:
sqlQuery = 'SELECT year, SUM(flight_count) AS year_count FROM sortedFlightData GROUP BY year'
spark.sql(sqlQuery).createOrReplaceTempView("TPY")

title("TPY")
spark.sql("SELECT * FROM TPY").show(10)

'---- TPY -----'
+----+----------+
|year|year_count|
+----+----------+
|2012|    385290|
|2014|    398022|
|2013|    381039|
|2011|    390663|
|2015|    411966|
|2010|    385450|
+----+----------+



### Exepected output for step  9
<pre>
'---- TPY -----'
+----+----------+
|year|year_count|
+----+----------+
|2012|    385290|
|2014|    398022|
|2013|    381039|
|2011|    390663|
|2015|    411966|
|2010|    385450|
+----+----------+
 </pre>

# 10. Use the above sums to compute percent Per Year, percent from Total for each country, year pair <font color="red">(CODE Needed)</font>
- Hint: make 1 `join` transformations

In [ ]:
totalFlightCount = spark.sql("SELECT * FROM sumAllTrips").first()[0]

sqlQuery = #CODE_HERE

spark.sql(sqlQuery).createOrReplaceTempView("stats")

title("stats count")
spark.sql("SELECT count(*) FROM stats").show()

title("stats show(10)")
spark.sql("SELECT * FROM stats").show(10)

### Exepected output for step  10
<pre>
'---- stats count -----'
+--------+
|count(1)|
+--------+
|     772|
+--------+

'---- stats show(10) -----'
+-----------+----+------------+-------+----------+
|    Country|year|flight_count|  total|year_count|
+-----------+----+------------+-------+----------+
|Afghanistan|2011|           8|2352430|    390663|
|Afghanistan|2012|           5|2352430|    385290|
|Afghanistan|2010|          11|2352430|    385450|
|    Algeria|2015|           4|2352430|    411966|
|    Algeria|2014|           9|2352430|    398022|
|    Algeria|2013|           2|2352430|    381039|
|     Angola|2015|          15|2352430|    411966|
|     Angola|2014|          13|2352430|    398022|
|     Angola|2011|          13|2352430|    390663|
|     Angola|2012|          12|2352430|    385290|
+-----------+----+------------+-------+----------+
only showing top 10 rows
 </pre>

# 11. Transform the above table into the format below <font color="red">(CODE Needed)</font>
- Hint: use the function `percent`


In [ ]:
def percent(number, total):
    return 100.0* number/total

spark.udf.register("percent", percent)

sqlQuery = #CODE_HERE

Table = spark.sql(sqlQuery)

Table.createOrReplaceTempView("Table")

title("Table count")
spark.sql("SELECT count(*) FROM Table").show()

title("Table show(10)")
spark.sql("SELECT * FROM Table").show(10)

### Exepected output for step  11
<pre>
'---- Table count -----'
+--------+
|count(1)|
+--------+
|     772|
+--------+

'---- Table show(10) -----'
+-----------+----+------------+-------+----------+--------------------+--------------------+
|    Country|year|flight_count|  total|year_count|           PcntTotal|            PcntYear|
+-----------+----+------------+-------+----------+--------------------+--------------------+
|Afghanistan|2010|          11|2352430|    385450|4.676015864446551E-4|0.002853807238292...|
|Afghanistan|2011|           8|2352430|    390663|3.400738810506582...|0.002047800789939...|
|Afghanistan|2012|           5|2352430|    385290|2.125461756566614E-4|0.001297723792468...|
|    Algeria|2013|           2|2352430|    381039|8.501847026266456E-5|  5.2488065526101E-4|
|    Algeria|2014|           9|2352430|    398022|3.825831161819905...|0.002261181542728794|
|    Algeria|2015|           4|2352430|    411966|1.700369405253291...|9.709539136724876E-4|
|     Angola|2010|          14|2352430|    385450| 5.95129291838652E-4|0.003632118303281...|
|     Angola|2011|          13|2352430|    390663|5.526200567073197E-4|0.003327676283651...|
|     Angola|2012|          12|2352430|    385290|5.101108215759874E-4|0.003114537101923...|
|     Angola|2013|          12|2352430|    381039|5.101108215759874E-4| 0.00314928393156606|
+-----------+----+------------+-------+----------+--------------------+--------------------+
only showing top 10 rows
</pre>

# 12. Final output


In [160]:
Table.toPandas()

Country  year  flight_count    total  year_count  \
0            Afghanistan  2010            11  2352430      385450   
1            Afghanistan  2011             8  2352430      390663   
2            Afghanistan  2012             5  2352430      385290   
3                Algeria  2013             2  2352430      381039   
4                Algeria  2014             9  2352430      398022   
5                Algeria  2015             4  2352430      411966   
6                 Angola  2010            14  2352430      385450   
7                 Angola  2011            13  2352430      390663   
8                 Angola  2012            12  2352430      385290   
9                 Angola  2013            12  2352430      381039   
10                Angola  2014            13  2352430      398022   
11                Angola  2015            15  2352430      411966   
12              Anguilla  2010            21  2352430      385450   
13              Anguilla  2011            21  2352430      390663   
14              Anguilla  2012            19  2352430      385290   
15              Anguilla  2013            22  2352430      381039   
16              Anguilla  2014            34  2352430      398022   
17              Anguilla  2015            41  2352430      411966   
18   Antigua and Barbuda  2010           123  2352430      385450   
19   Antigua and Barbuda  2011           146  2352430      390663   
20   Antigua and Barbuda  2012           145  2352430      385290   
21   Antigua and Barbuda  2013           123  2352430      381039   
22   Antigua and Barbuda  2014           115  2352430      398022   
23   Antigua and Barbuda  2015           126  2352430      411966   
24             Argentina  2010           184  2352430      385450   
25             Argentina  2011           183  2352430      390663   
26             Argentina  2012           208  2352430      385290   
27             Argentina  2013           176  2352430      381039   
28             Argentina  2014           195  2352430      398022   
29             Argentina  2015           180  2352430      411966   
..                   ...   ...           ...      ...         ...   
742       United Kingdom  2010          1629  2352430      385450   
743       United Kingdom  2011          1726  2352430      390663   
744       United Kingdom  2012          1852  2352430      385290   
745       United Kingdom  2013          1802  2352430      381039   
746       United Kingdom  2014          1912  2352430      398022   
747       United Kingdom  2015          2025  2352430      411966   
748        United States  2010        348113  2352430      385450   
749        United States  2011        352742  2352430      390663   
750        United States  2012        347452  2352430      385290   
751        United States  2013        343132  2352430      381039   
752        United States  2014        358354  2352430      398022   
753        United States  2015        370002  2352430      411966   
754              Uruguay  2010            54  2352430      385450   
755              Uruguay  2011            50  2352430      390663   
756              Uruguay  2012            53  2352430      385290   
757              Uruguay  2013            57  2352430      381039   
758              Uruguay  2014            60  2352430      398022   
759              Uruguay  2015            43  2352430      411966   
760            Venezuela  2010           377  2352430      385450   
761            Venezuela  2011           373  2352430      390663   
762            Venezuela  2012           389  2352430      385290   
763            Venezuela  2013           350  2352430      381039   
764            Venezuela  2014           335  2352430      398022   
765            Venezuela  2015           290  2352430      411966   
766              Vietnam  2010             1  2352430      385450   
767              Vietnam  2011             1  2352430      390663   
768            